##### Gil Levi and Tal Hassner.Age and Gender Classification Using Convolutional Neural Networks. IEEE Workshop on Analysis and Modeling of Faces and Gestures (AMFG), at the IEEE Conf. on Computer Vision and Pattern Recognition (CVPR), Boston, 2015.

https://talhassner.github.io/home/publication/2015_CVPR

In [0]:
!wget https://github.com/GilLevi/AgeGenderDeepLearning/raw/master/models/age_net.caffemodel
!wget https://github.com/GilLevi/AgeGenderDeepLearning/raw/master/models/gender_net.caffemodel
!wget https://github.com/GilLevi/AgeGenderDeepLearning/raw/master/models/mean.binaryproto
!wget https://raw.githubusercontent.com/GilLevi/AgeGenderDeepLearning/master/gender_net_definitions/deploy.prototxt
!mv deploy.prototxt gender_deploy.prototxt
!wget https://raw.githubusercontent.com/GilLevi/AgeGenderDeepLearning/master/age_net_definitions/deploy.prototxt
!mv deploy.prototxt age_deploy.prototxt
!apt install caffe-cpu
!wget https://upload.wikimedia.org/wikipedia/commons/8/8d/President_Barack_Obama.jpg

In [0]:
import caffe
import os
import numpy as np
import skimage.io

In [0]:
mean_filename='./mean.binaryproto'
proto_data = open(mean_filename, "rb").read()
a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data)
mean  = caffe.io.blobproto_to_array(a)[0]
print(mean.shape)

In [0]:
age_net_pretrained='./age_net.caffemodel'
age_net_model_file='./age_deploy.prototxt'
age_net = caffe.Classifier(age_net_model_file, age_net_pretrained,
                       mean=mean,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

In [0]:
gender_net_pretrained='./gender_net.caffemodel'
gender_net_model_file='./gender_deploy.prototxt'
gender_net = caffe.Classifier(gender_net_model_file, gender_net_pretrained,
                       mean=mean,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

In [0]:
age_list=['(0, 2)','(4, 6)','(8, 12)','(15, 20)','(25, 32)','(38, 43)','(48, 53)','(60, 100)']
gender_list=['Male','Female']

In [6]:
def load_image(filename, color=True):
    img = skimage.img_as_float(skimage.io.imread(filename, as_gray=not color)).astype(np.float32)
    if img.ndim == 2:
        img = img[:, :, np.newaxis]
        if color:
            img = np.tile(img, (1, 1, 3))
    elif img.shape[2] == 4:
        img = img[:, :, :3]
    return img

img = './President_Barack_Obama.jpg'
input_image = load_image(img)
print(input_image.shape)

(3356, 2687, 3)


In [7]:
prediction = age_net.predict([input_image]) 
print ('Predicted Age Group:', age_list[prediction[0].argmax()])
print('Probability: ', prediction[0][prediction[0].argmax()])

Predicted Age Group: (38, 43)
Probability:  0.6258768


In [0]:
prediction = gender_net.predict([input_image]) 
print ('Predicted Gender:', gender_list[prediction[0].argmax()])
print('Probability: ', prediction[0][prediction[0].argmax()])

Predicted Gender: Male
Probability:  0.7262794
